# Skincare Recommendation System

Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import heapq


Mount Google Drive

In [ ]:
#mount google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### Preparing Dataset

In [ ]:
skincare_df = pd.read_csv("/content/drive/MyDrive/CAPSTONE/products.csv")

In [ ]:
skincare_df.head()

,product_href,product_name,product_type,brand,notable_effects,skintype,price,description,picture_src
0,https://www.beautyhaul.com/product/detail/bubb...,ACWELL Bubble Free PH Balancing Cleanser,Face Wash,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,Rp 209.000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...
1,https://www.sociolla.com/face-wash/62452-ph-ba...,ACWELL pH Balancing Soothing Cleansing Foam,Face Wash,ACWELL,"Soothing, Balancing","Normal, Dry, Combination",Rp 181.800,Membersihkan dan menenangkan kulit sensitif de...,https://images.soco.id/8f08ced0-344d-41f4-a15e...
2,https://www.sociolla.com/toner/15871-licorice-...,Acwell Licorice pH Balancing Cleansing Toner,Toner,ACWELL,"Soothing, Balancing","Normal, Dry, Oily, Combination, Sensitive",Rp 149.000,"Mengangkat sisa kotoran, debu, dan make up sek...","https://www.soco.id/cdn-cgi/image/w=73,format=..."
3,https://www.beautyhaul.com/product/detail/aqua...,ACWELL Aquaseal Soothing Tonic,Toner,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,Rp 290.000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/assets/uploads/prod...
4,https://www.sociolla.com/essence/38023-licoric...,Licorice pH Balancing Essence Mist,Toner,ACWELL,"Brightening, Soothing","Normal, Dry",Rp 194.650,Essens mist dengan kelembapan tinggi yang memb...,"https://www.sociolla.com/cdn-cgi/image/w=425,f..."


In [ ]:
skincare_df['concern1'] = skincare_df['notable_effects'].str.split(',').str[0].str.strip()
skincare_df['concern2'] = skincare_df['notable_effects'].str.split(',').str[1].str.strip()
skincare_df['concern3'] = skincare_df['notable_effects'].str.split(',').str[2].str.strip()
skincare_df['concern4'] = skincare_df['notable_effects'].str.split(',').str[3].str.strip()

skincare_df.head()

,product_href,product_name,product_type,brand,notable_effects,skintype,price,description,picture_src,concern1,concern2,concern3,concern4
0,https://www.beautyhaul.com/product/detail/bubb...,ACWELL Bubble Free PH Balancing Cleanser,Face Wash,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,Rp 209.000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...,Acne-Free,Pore-Care,Brightening,Anti-Aging
1,https://www.sociolla.com/face-wash/62452-ph-ba...,ACWELL pH Balancing Soothing Cleansing Foam,Face Wash,ACWELL,"Soothing, Balancing","Normal, Dry, Combination",Rp 181.800,Membersihkan dan menenangkan kulit sensitif de...,https://images.soco.id/8f08ced0-344d-41f4-a15e...,Soothing,Balancing,NaN,NaN
2,https://www.sociolla.com/toner/15871-licorice-...,Acwell Licorice pH Balancing Cleansing Toner,Toner,ACWELL,"Soothing, Balancing","Normal, Dry, Oily, Combination, Sensitive",Rp 149.000,"Mengangkat sisa kotoran, debu, dan make up sek...","https://www.soco.id/cdn-cgi/image/w=73,format=...",Soothing,Balancing,NaN,NaN
3,https://www.beautyhaul.com/product/detail/aqua...,ACWELL Aquaseal Soothing Tonic,Toner,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,Rp 290.000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/assets/uploads/prod...,Acne-Free,Pore-Care,Brightening,Anti-Aging
4,https://www.sociolla.com/essence/38023-licoric...,Licorice pH Balancing Essence Mist,Toner,ACWELL,"Brightening, Soothing","Normal, Dry",Rp 194.650,Essens mist dengan kelembapan tinggi yang memb...,"https://www.sociolla.com/cdn-cgi/image/w=425,f...",Brightening,Soothing,NaN,NaN


In [ ]:
skincare_df = skincare_df[skincare_df['skintype'].isna() == False]
skincare_df.index = [i for i in range(0, len(skincare_df))]
skincare_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1224 entries, 0 to 1223
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   product_href     1224 non-null   object
 1   product_name     1224 non-null   object
 2   product_type     1224 non-null   object
 3   brand            1224 non-null   object
 4   notable_effects  1224 non-null   object
 5   skintype         1224 non-null   object
 6   price            1224 non-null   object
 7   description      1224 non-null   object
 8   picture_src      1224 non-null   object
 9   concern1         1224 non-null   object
 10  concern2         1035 non-null   object
 11  concern3         542 non-null    object
 12  concern4         79 non-null     object
dtypes: object(13)
memory usage: 133.9+ KB


In [ ]:
skincare_df['product_type'].value_counts()

,count
product_type,
Serum,307
Toner,255
Moisturizer,248
Sunscreen,212
Face Wash,202


In [ ]:
unique_effects = []
for effect_list in skincare_df['notable_effects'].str.split(',').dropna():
  if effect_list is not None:
    for effect in effect_list:
      effect = effect.strip()
      if effect not in unique_effects:
        unique_effects.append(effect)

unique_effects

['Acne-Free',
 'Pore-Care',
 'Brightening',
 'Anti-Aging',
 'Soothing',
 'Balancing',
 'Moisturizing',
 'Hydrating',
 'UV-Protection',
 'Oil-Control',
 'No-Whitecast',
 'Skin-Barrier',
 'Refreshing',
 'Black-Spot',
 'Oil-control',
 'Acne-Spot']

In [ ]:
unique_skintypes = []
for skin_list in skincare_df['skintype'].str.split(',').dropna():
  if skin_list is not None:
    for skin in skin_list:
      skin = skin.strip()
      if skin not in unique_skintypes:
        unique_skintypes.append(skin)

unique_skintypes

['Oily', 'Normal', 'Dry', 'Combination', 'Sensitive']

In [ ]:
skintypes_count = {}
for skin in unique_skintypes:
  skintypes_count[skin] = 0
  for skin_list in skincare_df['skintype'].str.split(','):
    if skin_list is not None:
      for current_skin in skin_list:
        current_skin = current_skin.strip()
        if current_skin == skin:
          skintypes_count[skin] += 1
          break

for skin, count in skintypes_count.items():
  print(f"{skin}: {count}")

Oily: 839
Normal: 538
Dry: 730
Combination: 441
Sensitive: 500


In [ ]:
EFFECTS = list(skincare_df.concern1.unique())
print(EFFECTS)

['Acne-Free', 'Soothing', 'Brightening', 'Moisturizing', 'Hydrating', 'Pore-Care', 'Anti-Aging', 'Balancing', 'UV-Protection', 'Skin-Barrier', 'Refreshing', 'Oil-Control']


In [ ]:
PRODUCTS = list(skincare_df.product_type.unique())
print(PRODUCTS)

['Face Wash', 'Toner', 'Serum', 'Moisturizer', 'Sunscreen']


In [ ]:
skincare_df[skincare_df['product_type'] ==  'Face Wash']['concern1'].value_counts()

,count
concern1,
Acne-Free,45
Hydrating,44
Moisturizing,34
Soothing,22
Pore-Care,19
Anti-Aging,17
Refreshing,7
Brightening,5
Balancing,4


In [ ]:
skincare_df[skincare_df['product_type'] == 'Sunscreen']['concern1'].value_counts()

,count
concern1,
UV-Protection,57
Brightening,33
Anti-Aging,32
Moisturizing,29
Hydrating,23
Soothing,16
Acne-Free,12
Skin-Barrier,5
Pore-Care,5


In [ ]:
skincare_df['notable_effects'] = skincare_df['notable_effects'].str.lower()

In [ ]:
top_concerns = {'Face Wash':'Refreshing', 'Toner':'Soothing', 'Serum':'Brightening', 'Moisturizer':'Moisturizing', 'Sunscreen':'UV-Protection' }
entries = len(skincare_df)
for i in range(entries):
    product_type = skincare_df.iloc[i]['product_type']
    if pd.isnull(skincare_df.iloc[i]['notable_effects']):
        skincare_df.iloc[i]['notable_effects'] = top_concerns[product_type]
skincare_df.info()



<class 'pandas.core.frame.DataFrame'>
Index: 1224 entries, 0 to 1223
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   product_href     1224 non-null   object
 1   product_name     1224 non-null   object
 2   product_type     1224 non-null   object
 3   brand            1224 non-null   object
 4   notable_effects  1224 non-null   object
 5   skintype         1224 non-null   object
 6   price            1224 non-null   object
 7   description      1224 non-null   object
 8   picture_src      1224 non-null   object
 9   concern1         1224 non-null   object
 10  concern2         1035 non-null   object
 11  concern3         542 non-null    object
 12  concern4         79 non-null     object
dtypes: object(13)
memory usage: 133.9+ KB


In [ ]:
skincare_df['brand'] = skincare_df['brand'].str.lower()
skincare_df['product_name'] = skincare_df['product_name'].str.lower()
skincare_df['skintype'] = skincare_df['skintype'].str.lower()
skincare_df['notable_effects'] = skincare_df['notable_effects'].str.replace(' and ', ',').str.replace(' or ', ',')
skincare_df

,product_href,product_name,product_type,brand,notable_effects,skintype,price,description,picture_src,concern1,concern2,concern3,concern4
0,https://www.beautyhaul.com/product/detail/bubb...,acwell bubble free ph balancing cleanser,Face Wash,acwell,"acne-free, pore-care, brightening, anti-aging",oily,Rp 209.000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...,Acne-Free,Pore-Care,Brightening,Anti-Aging
1,https://www.sociolla.com/face-wash/62452-ph-ba...,acwell ph balancing soothing cleansing foam,Face Wash,acwell,"soothing, balancing","normal, dry, combination",Rp 181.800,Membersihkan dan menenangkan kulit sensitif de...,https://images.soco.id/8f08ced0-344d-41f4-a15e...,Soothing,Balancing,NaN,NaN
2,https://www.sociolla.com/toner/15871-licorice-...,acwell licorice ph balancing cleansing toner,Toner,acwell,"soothing, balancing","normal, dry, oily, combination, sensitive",Rp 149.000,"Mengangkat sisa kotoran, debu, dan make up sek...","https://www.soco.id/cdn-cgi/image/w=73,format=...",Soothing,Balancing,NaN,NaN
3,https://www.beautyhaul.com/product/detail/aqua...,acwell aquaseal soothing tonic,Toner,acwell,"acne-free, pore-care, brightening, anti-aging",oily,Rp 290.000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/assets/uploads/prod...,Acne-Free,Pore-Care,Brightening,Anti-Aging
4,https://www.sociolla.com/essence/38023-licoric...,licorice ph balancing essence mist,Toner,acwell,"brightening, soothing","normal, dry",Rp 194.650,Essens mist dengan kelembapan tinggi yang memb...,"https://www.sociolla.com/cdn-cgi/image/w=425,f...",Brightening,Soothing,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,https://www.sociolla.com/face-cream-lotion/263...,whitelab brightening day cream,Moisturizer,whitelab,"brightening, uv-protection","normal, dry, oily, combination, sensitive",Rp 61.950,Whitelab Brightening Day Cream Fungsi: - Memba...,"https://www.soco.id/cdn-cgi/image/w=73,format=...",Brightening,UV-Protection,NaN,NaN
1220,https://shopee.co.id/YOU-The-Radiance-White-Pu...,you the radiance white purifying facial foam,Face Wash,you,"moisturizing, refreshing","normal, dry, oily, sensitive",Rp 52.000,YOU The Radiance White Purifying Facial Foam [...,https://cdn.istyle.im/images/product/web/27/61...,Moisturizing,Refreshing,NaN,NaN
1221,https://shopee.co.id/YOU-Basic-Skin-Care-Simpl...,you basic skin care simply fresh and bright fa...,Face Wash,you,"refreshing, acne-free","normal, dry, oily, sensitive",Rp 25.000,YOU Simply Fresh Honey and lemon Facial Wash S...,https://images.tokopedia.net/img/cache/700/pro...,Refreshing,Acne-Free,NaN,NaN
1222,https://shopee.co.id/YOU-Dazzling-Glow-Up-Faci...,you dazzling glow up facial foam,Face Wash,you,"hydrating, moisturizing","normal, dry, oily, sensitive",Rp 35.000,"A mild facial foam with Rice Extract, Tranexam...",https://images.tokopedia.net/img/cache/700/Vqb...,Hydrating,Moisturizing,NaN,NaN


### Concerns & Features

In [ ]:
concerns = ['acne', 'wrinkles', 'enlarged Pores', 'blackheads', 'redness', 'darkspot']
print(concerns)

['acne', 'wrinkles', 'enlarged Pores', 'blackheads', 'redness', 'darkspot']


In [ ]:
features = list(unique_skintypes) + list(concerns)
print(features)

features = ['Oily', 'Normal', 'Dry', 'Combination', 'Sensitive', 'acne', 'wrinkles', 'enlarged pores', 'blackheads', 'redness', 'darkspot']

['Oily', 'Normal', 'Dry', 'Combination', 'Sensitive', 'acne', 'wrinkles', 'enlarged Pores', 'blackheads', 'redness', 'darkspot']


### Sistem Rekomendasi

In [ ]:
# Step 1: Define concern mapping
concern_mapping = {
    'acne': ['Acne-Free', 'Acne-Spot'],
    'blackheads': ['Pore-Care', 'Oil-Control'],
    'enlarged pores': ['Pore-Care'],
    'darkspot': ['Brightening', 'Black-Spot'],
    'wrinkles': ['Anti-Aging'],
    'redness': ['Soothing', 'Balancing', 'Refreshing'],
    'Dry': ['Moisturizing', 'Hydrating'],
    'Combination': ['Moisturizing', 'Skin-Barrier'],
    'Oily': ['Oil-Control', 'Oil-control'],
    'Normal': ['Hydrating', 'Skin-Barrier'],
    'Sensitive': ['Skin-Barrier']
}

In [ ]:
def filter_products(row):
    # Memeriksa apakah skin type cocok
    product_skintypes = set(row['skintype'].split(', '))
    if not any(st.lower() in input_skin_condition for st in product_skintypes):
        return False  # Skin type tidak cocok

    # Memeriksa apakah semua efek yang direkomendasikan terpenuhi
    product_effects = {effect.lower() for effect in row['notable_effects'].split(', ')}
    return recommended_effects.issubset(product_effects)


In [ ]:
def recommend_products(input_conditions, skincare_df, concern_mapping):
    # Extract skin type and concerns from input
    skin_type = input_conditions[0].lower()
    concerns = {cond.lower() for cond in input_conditions[1:]}

    # Filter based on skin type
    filtered_df = skincare_df[skincare_df['skintype'].str.contains(skin_type, case=False, na=False)]

    # Generate a list of recommended effects based on concerns
    recommended_effects = set()
    for concern in concerns:
        if concern in concern_mapping:
            recommended_effects.update({effect.lower() for effect in concern_mapping[concern]})

    # Function to filter products based on notable effects
    def match_effects(effects):
        product_effects = {effect.lower() for effect in effects.split(', ')}
        return not recommended_effects.isdisjoint(product_effects)

    # Further filter the DataFrame based on notable effects
    filtered_df = filtered_df[filtered_df['notable_effects'].apply(match_effects)]

    # Select one product per product type
    representative_products = (
        filtered_df.groupby('product_type', as_index=False)
        .apply(lambda group: group.iloc[0])  # Take the first product in each group
        .reset_index(drop=True)
    )

    # Columns to display in the output
    output_columns = ['product_name', 'brand', 'product_type', 'price', 'skintype',
                      'concern1', 'concern2', 'concern3', 'concern4']

    return representative_products[output_columns]

In [ ]:
test_input = ['Dry', 'acne', 'darkspot']
recommendations = recommend_products(test_input, skincare_df, concern_mapping)
print(recommendations if not recommendations.empty else "No products found.")


                                    product_name        brand product_type  \
0        avoskin natural sublime facial cleanser      avoskin    Face Wash   
1          licorice ph balancing essential cream      acwell   Moisturizer   
2            licorice ph balancing advance serum      acwell         Serum   
3  banana boat ultra protect faces lotion spf50   banana boat    Sunscreen   
4             licorice ph balancing essence mist      acwell         Toner   

        price                                   skintype      concern1  \
0  Rp 109.000             normal, dry, oily, combination      Soothing   
1  Rp 304.000  normal, dry, oily, combination, sensitive  Moisturizing   
2  Rp 187.000             normal, dry, oily, combination  Moisturizing   
3  Rp 129.800                                        dry   Brightening   
4  Rp 194.650                                normal, dry   Brightening   

      concern2       concern3  concern4  
0    Balancing    Brightening       NaN  
1 

<ipython-input-52-2701311572a7>:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.iloc[0])  # Take the first product in each group


### fastAPI

In [ ]:
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.6 MB/s eta 0:00:00


In [ ]:
!pip install uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.4 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List

print("FastAPI and dependencies are installed successfully!")

FastAPI and dependencies are installed successfully!


In [ ]:
# Initialize FastAPI
app = FastAPI()

In [ ]:
# Define input model
class SkinCondition(BaseModel):
    conditions: List[str]  # List kondisi kulit pengguna

In [ ]:
# Recommendation function
def recommend_products(input_conditions: List[str], skincare_df: pd.DataFrame):
    # Generate recommended effects
    recommended_effects = set()
    for condition in input_conditions:
        if condition in concern_mapping:
            recommended_effects.update(concern_mapping[condition])

    # Filter products based on notable effects
    def filter_products(row):
        product_effects = set(row['notable_effects'].split(', '))
        return not recommended_effects.isdisjoint(product_effects)

    filtered_products = skincare_df[skincare_df.apply(filter_products, axis=1)]

     # Select relevant columns for display
    output_columns = ['product_type', 'product_name', 'brand', 'price', 'description', 'product_href']
    return filtered_products[output_columns].to_dict(orient="records")


In [ ]:
# Define API endpoint
@app.post("/recommend")
def get_recommendations(skin_condition: SkinCondition):
    recommendations = recommend_products(skin_condition.conditions, skincare_df)
    if not recommendations:
        return {"message": "No products found for the given skin conditions."}
    return {"recommendations": recommendations}

**Keterangan**

1. FastAPI Setup:
API memiliki satu endpoint utama `/recommend` yang menerima input berupa kondisi kulit.


2. Input Data:
Input berupa JSON dengan format
```
{
  "conditions": ["Oily", "Sensitive", "acne"]
}
```

3. Output Data:
API mengembalikan rekomendasi dalam format JSON
```
{
  "recommendations": [
    {
      "product_type": "Face Wash",
      "product_name": "ACWELL Bubble Free PH Balancing Cleanser",
      "brand": "ACWELL",
      "price": "Rp 209.000",
      "description": "Mengangkat kotoran dan menghapus makeup dalam satu langkah.",
      "product_href": "https://link-to-product.com"
    },
    ...
  ]
}
```

3. Logika Filter Produk:

Produk dipilih berdasarkan kecocokan antara efek yang direkomendasikan `(recommended_effects)` dan efek produk `(notable_effects)`.



# ERROR PART 1


In [ ]:
# Step 1: Define concern mapping
concern_mapping = {
    'acne': ['Acne-Free', 'Acne-Spot'],
    'comedo': ['Pore-Care', 'Oil-Control'],
    'pores': ['Pore-Care'],
    'darkspot': ['Brightening', 'Black-Spot'],
    'wrinkles': ['Anti-Aging'],
    'redness': ['Soothing', 'Balancing', 'Refreshing'],
    'Dry': ['Moisturizing', 'Hydrating'],
    'Combination': ['Moisturizing'],
    'Oily': ['Oil-Control', 'Oil-control'],
    'Normal': ['Hydrating', 'Skin-Barrier'],
    'Sensitive': ['Skin-Barrier']
}

In [ ]:
# Step 2: Prepare the features (One-Hot Encoding)
# List of unique features (skin types + concerns)
features = ['Oily', 'Normal', 'Dry', 'Combination', 'Sensitive',
            'acne', 'comedo', 'pores', 'redness', 'wrinkles', 'darkspot']

# Create an empty DataFrame for one-hot encoding
encoded_features = pd.DataFrame(0, index=skincare_df.index, columns=features)

# Populate one-hot encoding based on skintype column
for index, row in skincare_df.iterrows():
    # Split skintype into individual values and set them to 1
    for skintype in row['skintype'].split(', '):
        if skintype in features:
            encoded_features.at[index, skintype] = 1


In [ ]:
# Step 3: Input skin condition
input_skin_condition = ['Oily', 'comedo', 'darkspot']

# Generate a one-hot encoded vector for the input skin condition
input_vector = [1 if feature in input_skin_condition else 0 for feature in features]

In [ ]:
# Step 4: Map input skin condition to notable effects
recommended_effects = set()
for condition in input_skin_condition:
    if condition in concern_mapping:
        recommended_effects.update(concern_mapping[condition])

In [ ]:
# Step 5: Filter products based on notable effects
def filter_products(row):
    product_effects = set(row['notable_effects'].split(', '))
    return not recommended_effects.isdisjoint(product_effects)

recommended_products = skincare_df[skincare_df.apply(filter_products, axis=1)]

In [ ]:
# Step 6: Output the recommended products
output_columns = ['product_type', 'product_name', 'brand', 'price', 'description', 'product_href']
recommended_products_output = recommended_products[output_columns]


In [ ]:
recommended_products_output.head()

,product_type,product_name,brand,price,description,product_href
0,Face Wash,ACWELL Bubble Free PH Balancing Cleanser,ACWELL,Rp 209.000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/product/detail/bubb...
3,Toner,ACWELL Aquaseal Soothing Tonic,ACWELL,Rp 290.000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/product/detail/aqua...
4,Toner,Licorice pH Balancing Essence Mist,ACWELL,Rp 194.650,Essens mist dengan kelembapan tinggi yang memb...,https://www.sociolla.com/essence/38023-licoric...
5,Serum,Licorice pH Balancing Advance Serum,ACWELL,Rp 187.000,Serum wajah yang mengandung ekstrak akar licor...,https://www.sociolla.com/face-serum/54021-lico...
6,Moisturizer,Licorice pH Balancing Essential Cream,ACWELL,Rp 304.000,Moisturize dengan kandungan Licorice Roots yan...,https://www.sociolla.com/face-cream-lotion/716...


In [ ]:
# Step 1: Define concern mapping
concern_mapping = {
    'acne': ['Acne-Free', 'Acne-Spot'],
    'comedo': ['Pore-Care', 'Oil-Control'],
    'pores': ['Pore-Care'],
    'darkspot': ['Brightening', 'Black-Spot'],
    'wrinkles': ['Anti-Aging'],
    'redness': ['Soothing', 'Balancing', 'Refreshing'],
    'Dry': ['Moisturizing', 'Hydrating'],
    'Combination': ['Moisturizing'],
    'Oily': ['Oil-Control', 'Oil-control'],
    'Normal': ['Hydrating', 'Skin-Barrier'],
    'Sensitive': ['Skin-Barrier']
}

# Step 2: Prepare the features (One-Hot Encoding)
# List of unique features (skin types + concerns)
features = ['Oily', 'Normal', 'Dry', 'Combination', 'Sensitive',
            'acne', 'comedo', 'pores', 'redness', 'wrinkles', 'darkspot']

# Create an empty DataFrame for one-hot encoding
encoded_features = pd.DataFrame(0, index=skincare_df.index, columns=features)

# Populate one-hot encoding based on skintype column
for index, row in skincare_df.iterrows():
    # Split skintype into individual values and set them to 1
    for skintype in row['skintype'].split(', '):
        if skintype in features:
            encoded_features.at[index, skintype] = 1

# Step 3: Input skin condition
input_skin_condition = ['Dry', 'Sensitive', 'wrinkles', 'acne', 'darkspot']

# Generate a one-hot encoded vector for the input skin condition
input_vector = [1 if feature in input_skin_condition else 0 for feature in features]

# Step 4: Map input skin condition to notable effects
recommended_effects = set()
for condition in input_skin_condition:
    if condition in concern_mapping:
        recommended_effects.update(concern_mapping[condition])

# Step 5: Filter products based on notable effects
def filter_products(row):
    product_effects = set(row['notable_effects'].split(', '))
    return not recommended_effects.isdisjoint(product_effects)

recommended_products = skincare_df[skincare_df.apply(filter_products, axis=1)]

# Step 6: Output the recommended products
output_columns = ['product_type', 'product_name', 'brand', 'price', 'description', 'product_href']
recommended_products_output = recommended_products[output_columns]

recommended_products_output.head()


,product_type,product_name,brand,price,description,product_href
0,Face Wash,ACWELL Bubble Free PH Balancing Cleanser,ACWELL,Rp 209.000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/product/detail/bubb...
3,Toner,ACWELL Aquaseal Soothing Tonic,ACWELL,Rp 290.000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/product/detail/aqua...
4,Toner,Licorice pH Balancing Essence Mist,ACWELL,Rp 194.650,Essens mist dengan kelembapan tinggi yang memb...,https://www.sociolla.com/essence/38023-licoric...
5,Serum,Licorice pH Balancing Advance Serum,ACWELL,Rp 187.000,Serum wajah yang mengandung ekstrak akar licor...,https://www.sociolla.com/face-serum/54021-lico...
6,Moisturizer,Licorice pH Balancing Essential Cream,ACWELL,Rp 304.000,Moisturize dengan kandungan Licorice Roots yan...,https://www.sociolla.com/face-cream-lotion/716...


#ERROR PART 2


In [ ]:
concern_mapping = {
    'acne': ['Acne-Free', 'Acne-Spot'],
    'comedo': ['Pore-Care', 'Oil-Control'],
    'pores': ['Pore-Care'],
    'darkspot': ['Brightening', 'Black-Spot'],
    'wrinkles': ['Anti-Aging'],
    'redness': ['Soothing', 'Balancing', 'Refreshing'],
    'Dry': ['Moisturizing', 'Hydrating'],
    'Combination': ['Moisturizing'],
    'Oily': ['Oil-Control', 'Oil-control'],
    'Normal': ['Hydrating', 'Skin-Barrier'],
    'Sensitive': ['Skin-Barrier']
}

In [ ]:
def search_concern(target, i):
    if target in skincare_df2.iloc[i]['notable_effect']:
        return True
    return False

one_hot_encodings = np.zeros([entries, len(features)])

#skin types first
for i in range(entries):
    for j in range(5):
        target = features[j]
        skintype = skincare_df2.iloc[i]['skintype']
        if skintype == 'all':
            one_hot_encodings[i][0:5] = 1
        elif target == skintype:
            one_hot_encodings[i][j] = 1

#other features
for i in range(entries):
    for j in range(5, len(features)):
        feature = features[j]
        if feature in skincare_df2.iloc[i]['concern1']:
            one_hot_encodings[i][j] = 1



In [ ]:
x = one_hot_encodings[456]

In [ ]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(one_hot_encodings)
distances, indices = nbrs.kneighbors(one_hot_encodings)


In [ ]:
# utility functions
def name2index(name):
    return skincare_df2[skincare_df2["product_name"]==name].index.tolist()[0]

def index2prod(index):
    return skincare_df2.iloc[index]

In [ ]:

def wrap(info_arr):
    result = {}
#   print(info_arr)
    result['brand'] = info_arr[0]
    result['product_name'] = info_arr[1]
    result['price'] = info_arr[2]
    result['product_href'] = info_arr[3]
    result['skintype'] = info_arr[4]
    result['notable_effects'] = str(info_arr[5]).split(',')
    return result


# recommend top 10 similar items from a category
def recs_cs(vector = None, product_name = None, product_type = None, count = 5):
    products = []
    if product_name:
        idx = name2index(product_name)
        fv = one_hot_encodings[idx]
    elif vector:
        fv = vector
    cs_values = cosine_similarity(np.array([fv, ]), one_hot_encodings)
    skincare_df2['cs'] = cs_values[0]

    if product_type:
        skincare_dff = skincare_df2[skincare_df2['product_type'] == product_type]
    else:
        skincare_dff = skincare_df2

    if product_name:
        skincare_dff = skincare_dff[skincare_dff['product_name'] != product_name]
    recommendations = skincare_dff.sort_values('cs', ascending=False).head(count)
    #   print(f"Top {count} matching {label} items")
    data = recommendations[['brand', 'product_name', 'price', 'product_href','skintype','notable_effects']].to_dict('split')['data']
    for element in data:
        products.append(wrap(element))
    return products

In [ ]:
# overall recommendation
def recs_essentials(vector = None, product_name = None):
#     print("ESSENTIALS:")
    response = {}
    for product in PRODUCTS:
#         print(f"{label}:")
        if product_name:
            r = recs_cs(None, product_name, product_type)
        elif vector:
            r = recs_cs(vector, None, product_type)
        else:
            r = recs_cs(None, None, product_type)
        response[product_type] = r
    return response


In [ ]:
x = [0,1,0,0,0,1,0,1,0,0,0]

y1 = recs_cs(x, None, 'Face Wash')
y2 = recs_cs(x, None, 'Toner')
y3 = recs_cs(x, None, 'Serum')
y4 = recs_cs(x, None, 'Moisturizer')
y5 = recs_cs(x, None, 'Sunscreen')

y1

[{'brand': 'acwell ',
  'product_name': 'acwell bubble free ph balancing cleanser',
  'price': 'Rp 209.000',
  'product_href': 'https://www.beautyhaul.com/product/detail/bubble-free-ph-balancing-cleanser',
  'skintype': 'oily',
  'notable_effects': ['acne-free',
   ' pore-care',
   ' brightening',
   ' anti-aging']},
 {'brand': 'sk-ii',
  'product_name': 'sk-ii cleansing gel 80gr',
  'price': 'Rp 540.000',
  'product_href': 'https://www.zalora.co.id/sk-ii-sk-ii-cleansing-gel-80gr-white-2975877.html',
  'skintype': 'oily, combination',
  'notable_effects': ['acne-free', ' oil-control', ' pore-care']},
 {'brand': 'senka',
  'product_name': 'senka perfect whip fresh anti shine',
  'price': 'Rp 56.000',
  'product_href': 'https://shopee.co.id/SENKA-Perfect-Whip-Fresh-Anti-Shine-100gr-i.138850166.2202583084?sp_atk=ca715a5d-657d-4292-975f-be981e2ce79a',
  'skintype': 'normal, dry, oily, sensitive',
  'notable_effects': ['acne-free', ' oil-control', ' pore-care']},
 {'brand': 'senka',
  'prod

In [ ]:
y2

[{'brand': 'acwell ',
  'product_name': 'acwell licorice ph balancing cleansing toner ',
  'price': 'Rp 149.000',
  'product_href': 'https://www.sociolla.com/toner/15871-licorice-ph-balancing-cleansing-toner',
  'skintype': 'normal, dry, oily, combination, sensitive',
  'notable_effects': ['soothing', ' balancing']},
 {'brand': "n'pure",
  'product_name': 'npure cica clear pad',
  'price': 'Rp 129.000',
  'product_href': 'https://www.beautyhaul.com/product/detail/cica-clear-pad',
  'skintype': 'dry, oily, sensitive',
  'notable_effects': ['brightening', ' anti-aging']},
 {'brand': 'natur',
  'product_name': 'natur miracle brightening hydrating toner',
  'price': 'Rp 139.000',
  'product_href': 'https://www.beautyhaul.com/product/detail/miracle-brightening-hydrating-toner',
  'skintype': 'oily',
  'notable_effects': ['pore-care', ' brightening', ' anti-aging']},
 {'brand': 'natur',
  'product_name': 'natur miracle calming hydrating toner',
  'price': 'Rp 139.000',
  'product_href': 'htt

In [ ]:

y3

[{'brand': 'acwell ',
  'product_name': 'licorice ph balancing advance serum',
  'price': 'Rp 187.000',
  'product_href': 'https://www.sociolla.com/face-serum/54021-licorice-ph-balancing-advance-serum ',
  'skintype': 'normal, dry, oily, combination',
  'notable_effects': ['moisturizing', ' brightening', ' anti-aging']},
 {'brand': 'skeyndor ',
  'product_name': 'skeyndor eternal intensive serum',
  'price': 'Rp 1.045.000',
  'product_href': 'https://www.beautyhaul.com/product/detail/eternal-intensive-serum',
  'skintype': 'normal, dry, oily, sensitive',
  'notable_effects': ['brightening', ' anti-aging']},
 {'brand': 'skin dewi',
  'product_name': 'skin dewi tamanu green serum ',
  'price': 'Rp 547.800',
  'product_href': 'https://www.beautyhaul.com/product/detail/tamanu-green-serum',
  'skintype': 'oily, sensitive',
  'notable_effects': ['soothing', ' acne-free']},
 {'brand': 'skeyndor',
  'product_name': 'skeyndor lift contour elixir - face and neck ',
  'price': 'Rp 1.595.000',
  '

In [ ]:
y4


[{'brand': 'acwell ',
  'product_name': 'licorice ph balancing essential cream',
  'price': 'Rp 304.000',
  'product_href': 'https://www.sociolla.com/face-cream-lotion/71655-licorice-ph-balancing-essential-cream',
  'skintype': 'normal, dry, oily, combination, sensitive',
  'notable_effects': ['moisturizing', ' brightening']},
 {'brand': 'purito',
  'product_name': 'purito sea buckthorn vital 70 cream ',
  'price': 'Rp 412.500',
  'product_href': 'https://www.sociolla.com/face-cream-lotion/25445-sea-buckthorn-vital-70-cream',
  'skintype': 'normal, dry, oily, combination, sensitive',
  'notable_effects': ['soothing', ' moisturizing']},
 {'brand': "n'pure",
  'product_name': 'npure day cream cica',
  'price': 'Rp 115.000',
  'product_href': 'https://www.beautyhaul.com/product/detail/day-cream-cica',
  'skintype': 'oily',
  'notable_effects': ['pore-care', ' brightening', ' anti-aging']},
 {'brand': "n'pure",
  'product_name': 'npure cactus aloe vera soothing gel',
  'price': 'Rp 80.000'

In [ ]:
y5

[{'brand': 'aizen ',
  'product_name': 'aizen sunorion uv sunscreen',
  'price': 'Rp 139.000',
  'product_href': 'https://www.beautyhaul.com/product/detail/sunorion-uv-sunscreen',
  'skintype': 'dry, oily, sensitive',
  'notable_effects': ['anti-aging', ' uv-protection']},
 {'brand': 'loreal paris',
  'product_name': 'loreal paris uv defender matte & fresh ',
  'price': 'Rp 319.000',
  'product_href': 'https://www.beautyhaul.com/product/detail/uv-defender-matte-fresh',
  'skintype': 'dry',
  'notable_effects': ['anti-aging', ' uv-protection']},
 {'brand': 'loreal paris',
  'product_name': 'loreal paris uv perfect transparent spf 50+ pa++++',
  'price': 'Rp 99.000',
  'product_href': 'https://www.beautyhaul.com/product/detail/uv-perfect-transparent-spf-50-pa',
  'skintype': 'normal, dry, oily, combination',
  'notable_effects': ['anti-aging',
   ' skin-barrier',
   ' uv-protection',
   ' no-whitecast']},
 {'brand': 'loreal paris',
  'product_name': "loreal paris l'oreal paris uv defende

In [ ]:
x = [1,0,0,0,0,0,1,1,0,0,0]

y = recs_essentials(x, None,)
y

{'Face Wash': [{'brand': 'acwell ',
   'product_name': 'acwell bubble free ph balancing cleanser',
   'price': 'Rp 209.000',
   'product_href': 'https://www.beautyhaul.com/product/detail/bubble-free-ph-balancing-cleanser',
   'skintype': 'oily',
   'notable_effects': ['acne-free',
    ' pore-care',
    ' brightening',
    ' anti-aging']},
  {'brand': 'sk-ii',
   'product_name': 'sk-ii cleansing gel 80gr',
   'price': 'Rp 540.000',
   'product_href': 'https://www.zalora.co.id/sk-ii-sk-ii-cleansing-gel-80gr-white-2975877.html',
   'skintype': 'oily, combination',
   'notable_effects': ['acne-free', ' oil-control', ' pore-care']},
  {'brand': 'senka',
   'product_name': 'senka perfect whip fresh anti shine',
   'price': 'Rp 56.000',
   'product_href': 'https://shopee.co.id/SENKA-Perfect-Whip-Fresh-Anti-Shine-100gr-i.138850166.2202583084?sp_atk=ca715a5d-657d-4292-975f-be981e2ce79a',
   'skintype': 'normal, dry, oily, sensitive',
   'notable_effects': ['acne-free', ' oil-control', ' pore-ca

In [ ]:
def recommend_products(input_conditions, skincare_df, concern_mapping, features):

    # Step 1: Generate recommended effects from input conditions
    recommended_effects = set()
    for condition in input_conditions:
        if condition in concern_mapping:
            recommended_effects.update(concern_mapping[condition])

    # Step 2: Filter products based on recommended effects
    def filter_products(row):
        product_effects = set(row['notable_effects'].split(', '))
        return not recommended_effects.isdisjoint(product_effects)

    filtered_products = skincare_df[skincare_df.apply(filter_products, axis=1)]

    # Step 3: Select relevant columns for display
    output_columns = ['product_type', 'product_name', 'brand', 'price', 'description', 'product_href']
    return filtered_products[output_columns]

# Test cases
test_inputs = [
    ['Oily', 'Sensitive', 'acne'],  # Test case 1
    ['Dry', 'wrinkles', 'darkspot'],  # Test case 2
    ['Combination', 'pores', 'redness'],  # Test case 3
    ['Normal'],  # Test case 4
]

# Execute test cases
for i, test_input in enumerate(test_inputs, start=1):
    print(f"Test Case {i}: Input Conditions = {test_input}")
    recommendations = recommend_products(test_input, skincare_df, concern_mapping, features)
    if recommendations.empty:
        print("No products found for this condition.")
    else:
        print(recommendations.head(), "\n")


Test Case 1: Input Conditions = ['Oily', 'Sensitive', 'acne']
   product_type                                       product_name     brand  \
0     Face Wash           ACWELL Bubble Free PH Balancing Cleanser   ACWELL    
3         Toner                    ACWELL Aquaseal Soothing Tonic    ACWELL    
12        Toner  AIZEN Incredipeel Exfoliating + Purifying Faci...    AIZEN    
19    Sunscreen             ANESSA Perfect UV Sunscreen Mild Milk    ANESSA    
25    Face Wash             A'pieu Deep Clean Foam Cleanser (Pore)  A'PIEU     

         price                                        description  \
0   Rp 209.000  Mengangkat kotoran dan menghapus makeup dalam ...   
3   Rp 290.000  Pre-essence yang diformulasikan dengan ekstrak...   
12  Rp 139.000  Incredipeel Exfoliating & Purifying Facial Ton...   
19  Rp 489.000  Anessa Perfect UV MILD Milk adalah Ultimate be...   
25   Rp 69.000  A'PIEU Pembersih wajah yang mengandung baking ...   

                                         p